# Louvain

The Louvain method is a simple, efficient and easy-to-implement method for identifying communities in large networks. The method unveils hierarchies of communities and allows to zoom within communities to discover sub-communities, sub-sub-communities, etc. It is one of the most widely used method for detecting communities in large networks. 

In [70]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
import os

In [71]:
host = os.environ.get("NEO4J_HOST", "bolt://localhost") 
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver(host, auth=basic_auth(user, password))

In [72]:
create_follows_graph_query = """\
MERGE (nAlice:User {id:'Alice'})
MERGE (nBridget:User {id:'Bridget'})
MERGE (nCharles:User {id:'Charles'})
MERGE (nDoug:User {id:'Doug'})
MERGE (nMark:User {id:'Mark'})
MERGE (nMichael:User {id:'Michael'})

MERGE (nAlice)-[:FOLLOW]->(nBridget)
MERGE (nAlice)-[:FOLLOW]->(nCharles)
MERGE (nMark)-[:FOLLOW]->(nDoug)
MERGE (nBridget)-[:FOLLOW]->(nMichael)
MERGE (nDoug)-[:FOLLOW]->(nMark)
MERGE (nMichael)-[:FOLLOW]->(nAlice)
MERGE (nAlice)-[:FOLLOW]->(nMichael)
MERGE (nBridget)-[:FOLLOW]->(nAlice)
MERGE (nMichael)-[:FOLLOW]->(nBridget)
MERGE (nCharles)-[:FOLLOW]->(nDoug)
"""

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_follows_graph_query))
    print("Stats: " + str(result.consume().metadata.get("stats", {})))

Stats: {}


In [73]:
streaming_query = """\
CALL algo.louvain.stream(
  'MATCH (u:User) RETURN id(u) as id', 
  'MATCH (u1:User)-[:FOLLOW]->(u2:User) 
   RETURN id(u1) as source, id(u2) as target', 
  { iterations:20, dampingFactor:0.85 })
YIELD nodeId, community
RETURN nodeId, community 
LIMIT 20
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(streaming_query))        
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
df    

,nodeId,community
0,0,5
1,1,5
2,2,5
3,3,3
4,4,3
5,5,5


In [74]:
write_results_query = """\
CALL algo.louvain(
  'MATCH (u:User) RETURN id(u) as id', 
  'MATCH (u1:User)-[:FOLLOW]->(u2:User) 
   RETURN id(u1) as source, id(u2) as target', 
  { write: true, writeProperty:"louvain" })
YIELD loadMillis, computeMillis, writeMillis, nodes, iterations, communityCount
"""

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(write_results_query))        
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())

df

,loadMillis,computeMillis,writeMillis,nodes,iterations,communityCount
0,0,0,0,6,2,2


In [75]:
user_louvain_query = """\
MATCH (user:User)
RETURN user.louvain AS louvain, COLLECT(user.id) AS users
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(user_louvain_query))
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
df

,louvain,users
0,5,"[Alice, Bridget, Charles, Michael]"
1,4,"[Doug, Mark]"
